# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233485269929                   -0.50    7.0
  2   -7.249375714169       -1.80       -1.40    1.0
  3   -7.251228986913       -2.73       -1.92    4.0
  4   -7.251174198183   +   -4.26       -2.04    3.0
  5   -7.251297557748       -3.91       -2.36    2.0
  6   -7.251336261366       -4.41       -2.89    2.0
  7   -7.251338746082       -5.60       -3.83    2.0
  8   -7.251338783728       -7.42       -3.96    5.0
  9   -7.251338797394       -7.86       -4.48    2.0
 10   -7.251338798611       -8.91       -5.06    3.0
 11   -7.251338798696      -10.07       -5.60    3.0
 12   -7.251338798704      -11.08       -6.23    3.0
 13   -7.251338798705      -12.50       -7.16    3.0
 14   -7.251338798705      -14.57       -7.71    4.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04527524592656215
[ Info: Arnoldi iteration step 2: normres = 0.3716427594861368
[ Info: Arnoldi iteration step 3: normres = 0.24317627967095765
[ Info: Arnoldi iteration step 4: normres = 0.6415640229283908
[ Info: Arnoldi iteration step 5: normres = 0.42492896690217674
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.45e-01, 8.72e-02, 1.65e-01, 1.50e-01, 6.45e-02)
[ Info: Arnoldi iteration step 6: normres = 0.8528726239550658
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.99e-01, 3.06e-01, 3.58e-01, 9.79e-02, 5.62e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08574047379946005
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.38e-02, 2.07e-02, 8.82e-03, 2.50e-02, 6.44e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09681155836991369
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.41e